# Python과 텔레그램 봇 API를 이용해서 챗봇 만들기 실습 - 완전 기초

* 싸이랭 / 싸이랭 : 파트 3 - Python을 이용한 자연어처리 기초 실습
* 김무성

# 0. 싸이랭/랭봇 docker image pulling & container 실행

실습 소스를 가져온다.

실습을 위한 docker 이미지 받는다.(docker 환경 자체는 구성되어 있어야 함)

docker 컨테이너를 실행시킨다. 그러기 위해서 소스의 docker-scripts 디렉토리로 이동한다. 
실행 스크립트 run_langbot.sh에서 []로 되어 있는 부분을 자신만의 내용으로 변경해준다.

스크립트 실행

웹 브라우저로 접속한다

# 1. @BotFather를 통한 봇 등록 / 토큰 발급 / 설정 

이제부터는 langbot/ui/telegram 디렉토리 안의 test_langbot_telegram.ipynb 파일 기준으로 진행된다.

### 텔레그램 봇 등록 & 토큰 발행

#### 참고
* [1] 텔레그램 로봇, 공식 API로 만들기 (파이썬, 구글 앱 엔진) - http://bakyeono.net/post/2015-08-24-using-telegram-bot-api.html

발급받은 토큰을 langbot.keyfile의 첫줄에 복사해넣는다.

In [8]:
!ls ./

langbot.keyfile   langbot_serve.py.old	test_langbot_telegram.ipynb
langbot_serve.py  supervisord.conf


# 2. python-telegram-bot 설치

#### 참고
* [2] python-telegram-bot - https://github.com/python-telegram-bot/python-telegram-bot

이 실습에서는 python3 버전으로 설치한다.

In [2]:
!pip3 install python-telegram-bot --upgrade

Downloading/unpacking python-telegram-bot
Downloading/unpacking urllib3>=1.10 (from python-telegram-bot)
Downloading/unpacking future>=0.15.2 (from python-telegram-bot)
  Running setup.py (path:/tmp/pip_build_root/future/setup.py) egg_info for package future
    
Downloading/unpacking certifi (from python-telegram-bot)
  Found existing installation: urllib3 1.7.1
    Not uninstalling urllib3 at /usr/lib/python3/dist-packages, owned by OS
  Running setup.py install for future
    
    Installing futurize script to /usr/local/bin
    Installing pasteurize script to /usr/local/bin
  Could not find .egg-info directory in install record for future>=0.15.2 (from python-telegram-bot)
Cleaning up...


# 3. 간단한 에코 봇 만들기 

#### 참고
* [3] Extensions – Your first Bot - https://github.com/python-telegram-bot/python-telegram-bot/wiki/Extensions-%E2%80%93-Your-first-Bot

In [3]:
import os

In [4]:
from telegram.ext import Updater
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters

In [5]:
with open(os.path.join('.','langbot.keyfile'), 'r') as f :
        token = f.readline().strip()

In [6]:
updater = Updater(token=token)

In [7]:
dispatcher = updater.dispatcher

### 예제 1) 시작 메시지 등록

In [8]:
def start(bot, update):
    bot.sendMessage(chat_id=update.message.chat_id, text="안녕, 난 언어학습을 도와주는 랭봇이야!")

In [9]:
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

### 예제 2) 유저 챗 아이디 체크하고, 확인하는 핸들러 등록

In [10]:
G_STATES = {}

In [11]:
def init_state(chat_id) :
    G_STATES[chat_id] = '' 

In [12]:
def check_state(chat_id) :
    if not chat_id in G_STATES:
        init_state(chat_id)

In [13]:
def get_state(chat_id) :
    return G_STATES[chat_id]

In [14]:
def set_state(chat_id, val) :
    G_STATES[chat_id] = val

In [15]:
def iam(bot, update):
    chat_id = update.message.chat_id
    check_state(chat_id)
    st = get_state(chat_id)
    msg = "my chat_id is %d" %(chat_id)
    msg = msg + "\n" + "my state is %s" %(st)
    bot.sendMessage(chat_id=update.message.chat_id, text=msg)
    set_state(chat_id, st+str(1)) # 이 부분은 유저의 상태를 핸들링할 수 있다는 것을 보기 위해 추가되었다.

In [16]:
iam_handler = CommandHandler('iam', iam)

In [17]:
dispatcher.add_handler(iam_handler)

### 예제 3) 에코 봇으로 만들기 (대답 따라하는 메시지 핸들러 등록)

In [18]:
def echo(bot, update):
    bot.sendMessage(chat_id=update.message.chat_id, text=update.message.text)

In [19]:
echo_handler = MessageHandler(Filters.text, echo)
dispatcher.add_handler(echo_handler)

### 봇 실행

In [20]:
updater.start_polling()

# 4. Supervisor를 이용해서 봇 스크립트를 계속 실행시키기

#### 참고자료
* [4] Building AI Chat bot using Python 3 & TensorFlow - https://speakerdeck.com/inureyes/building-ai-chat-bot-using-python-3-and-tensorflow
* [5] Supervisor: A Process Control System - http://supervisord.org/

### supervisor 설치

supervisor는 python2만 지원(아직은)

In [1]:
!pip install supervisor

/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:315: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
    100% |################################| 419kB 867kB/s 
  Running setup.py bdist_wheel for supervisor ... - \ done
  Stored in direc

### 스크립트에 포함된 패키지들 설치

bs4 패키지가 설치되어야 한다

In [6]:
!pip3 install bs4

Cleaning up...


### Supervisor 설정파일에 스크립트 등록

Supervisor에 봇 스크립트를 등록해서 실행시키기 위해서, supervisord.conf라는 파일에 몇가지 설정을 해야 한다. 그리고 특정 경로에 그 파일을 옮겨준다.

In [9]:
!ls

langbot.keyfile   langbot_serve.py.old	test_langbot_telegram.ipynb
langbot_serve.py  supervisord.conf


In [5]:
!cp supervisord.conf /etc/supervisord.conf

### Running Supervisor

터미널을 열고 (주피터 터미널을 쓰면 좀 더 쉬움) supervisor 명령 실행

그리고 다시 터미널에서 supervisorctl 명령 실행

# 참고자료 
* [1] 텔레그램 로봇, 공식 API로 만들기 (파이썬, 구글 앱 엔진) - http://bakyeono.net/post/2015-08-24-using-telegram-bot-api.html
* [2] python-telegram-bot - https://github.com/python-telegram-bot/python-telegram-bot
* [3] Extensions – Your first Bot - https://github.com/python-telegram-bot/python-telegram-bot/wiki/Extensions-%E2%80%93-Your-first-Bot
* [4] 
Building AI Chat bot using Python 3 & TensorFlow - https://speakerdeck.com/inureyes/building-ai-chat-bot-using-python-3-and-tensorflow
* [5] Supervisor: A Process Control System - http://supervisord.org/